In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
directory_list = os.listdir('/content/drive/My Drive')
print(directory_list)
all_items = os.listdir('/content/drive/My Drive')
print(all_items)

print("Current directory:", os.getcwd())
new_directory = '/content/drive/My Drive'
os.chdir(new_directory)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['small bird', 'Processed Small Bird', 'Filtered_output_folder (1)', 'Filtered_output_folder', 'output_folder', 'Untitled0.ipynb', 'compare_counter&mask.ipynb', 'LayoutCRF', 'Untitled1.ipynb']
['small bird', 'Processed Small Bird', 'Filtered_output_folder (1)', 'Filtered_output_folder', 'output_folder', 'Untitled0.ipynb', 'compare_counter&mask.ipynb', 'LayoutCRF', 'Untitled1.ipynb']
Current directory: /content/drive/My Drive


In [ ]:
# threshold = 30
import cv2
import os
import numpy as np

# Define the input and output directory paths
input_dir_path = '/content/drive/My Drive/LayoutCRF/SmallBird_Images/images'
processed_dir_path = '/content/drive/My Drive/Filtered_output_folder'
final_output_dir_path = '/content/drive/My Drive/output_folder/Three Consecutive output Contour'

# Create the necessary directories if they don't exist
for path in [processed_dir_path, final_output_dir_path]:
    os.makedirs(path, exist_ok=True)

# Process the image: Apply Gaussian Blur and Histogram Equalization
def process_image(image_path, output_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Image at {image_path} didn't load properly.")
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    enhanced = cv2.equalizeHist(blurred)
    cv2.imwrite(output_path, enhanced)

# Find common changes between three consecutive images
def find_common_changes(image_paths):
    images = [cv2.imread(path, cv2.IMREAD_GRAYSCALE) for path in image_paths]
    if any(img is None for img in images):
        raise ValueError("One of the images didn't load properly. Check the file paths.")
    deltas = [cv2.absdiff(images[i+1], images[i]) for i in range(2)]
    threshs = [cv2.threshold(delta, 30, 255, cv2.THRESH_BINARY)[1] for delta in deltas]
    common_changes = cv2.bitwise_and(threshs[0], threshs[1])
    return common_changes

# Apply morphological operations to reduce noise
def clean_changes(image):
    kernel = np.ones((3, 3), np.uint8)
    opened = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
    closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel)
    return closed

# Apply contour detection and draw on the cleaned changes
def detect_and_draw_contours(cleaned_changes, processed_image_path, output_path):
    image = cv2.imread(processed_image_path)
    if image is None:
        raise ValueError(f"Failed to load the processed image from {processed_image_path}")
    # Convert grayscale to BGR for color drawing
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    contours, _ = cv2.findContours(cleaned_changes, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > 150:  # Adjust this value as needed
            cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)  # Drawing in green
    cv2.imwrite(output_path, image)

# Get all the image file names from the input directory
input_image_files = [file for file in os.listdir(input_dir_path) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Process each image and save it to the processed directory
for file_name in input_image_files:
    input_image_path = os.path.join(input_dir_path, file_name)
    processed_image_path = os.path.join(processed_dir_path, 'processed_' + file_name)
    process_image(input_image_path, processed_image_path)

# Process in overlapping windows of three images
processed_image_files = sorted([f for f in os.listdir(processed_dir_path) if f.endswith(('png', 'JPG', 'jpeg'))])
if len(processed_image_files) < 3:
    raise ValueError("Not enough processed images to find common changes. Need at least three.")

for i in range(0, len(processed_image_files) - 2, 2):  # Step by 2 for overlapping groups
    image_paths = [os.path.join(processed_dir_path, processed_image_files[j]) for j in range(i, i + 3)]
    common_changes = find_common_changes(image_paths)
    cleaned_changes = clean_changes(common_changes)
    output_path = os.path.join(final_output_dir_path, f'common_changes_{i}.jpg')
    detect_and_draw_contours(cleaned_changes, image_paths[0], output_path)
    print(f"Processed images {i}, {i+1}, and {i+2}")


Processed images 0, 1, and 2
Processed images 2, 3, and 4
Processed images 4, 5, and 6
Processed images 6, 7, and 8
Processed images 8, 9, and 10
Processed images 10, 11, and 12
Processed images 12, 13, and 14
Processed images 14, 15, and 16
Processed images 16, 17, and 18
Processed images 18, 19, and 20
Processed images 20, 21, and 22
Processed images 22, 23, and 24
Processed images 24, 25, and 26
Processed images 26, 27, and 28
Processed images 28, 29, and 30
Processed images 30, 31, and 32
Processed images 32, 33, and 34
Processed images 34, 35, and 36
Processed images 36, 37, and 38
Processed images 38, 39, and 40
Processed images 40, 41, and 42
Processed images 42, 43, and 44
Processed images 44, 45, and 46
Processed images 46, 47, and 48
Processed images 48, 49, and 50
Processed images 50, 51, and 52
Processed images 52, 53, and 54
Processed images 54, 55, and 56
Processed images 56, 57, and 58
Processed images 58, 59, and 60
Processed images 60, 61, and 62
Processed images 62, 6